In [ ]:
import numpy as np
import pandas as pd

# Sample decision matrix (rows: models, columns: criteria)
# Replace with actual data for your task
data = {
    "Model": ["Model_A", "Model_B", "Model_C", "Model_D"],
    "Accuracy": [0.85, 0.88, 0.82, 0.87],  # Higher is better (+)
    "Inference_Time": [120, 100, 150, 110],  # Lower is better (-)
    "Model_Size": [500, 450, 600, 480],  # Lower is better (-)
    "F1_Score": [0.80, 0.83, 0.78, 0.81],  # Higher is better (+)
}

df = pd.DataFrame(data)


def normalize_matrix(matrix):
    norm_matrix = matrix.copy()
    for col in matrix.columns[1:]:
        norm_matrix[col] = matrix[col] / np.sqrt(sum(matrix[col]**2))
    return norm_matrix

norm_df = normalize_matrix(df)

# Define weights (customize as needed)
weights = np.array([0.4, 0.2, 0.2, 0.2])  # Adjust according to importance

# Identify benefit (+) and cost (-) criteria
benefit_criteria = ["Accuracy", "F1_Score"]
cost_criteria = ["Inference_Time", "Model_Size"]

# Compute weighted normalized matrix
weighted_matrix = norm_df.copy()
for i, col in enumerate(weighted_matrix.columns[1:]):
    weighted_matrix[col] = weighted_matrix[col] * weights[i]

# Determine ideal best and worst solutions
ideal_best = {col: max(weighted_matrix[col]) if col in benefit_criteria else min(weighted_matrix[col]) for col in weighted_matrix.columns[1:]}
ideal_worst = {col: min(weighted_matrix[col]) if col in benefit_criteria else max(weighted_matrix[col]) for col in weighted_matrix.columns[1:]}

# Calculate separation measures
def calculate_separation(matrix, ideal_solution):
    return np.sqrt(((matrix.iloc[:, 1:] - pd.Series(ideal_solution))**2).sum(axis=1))

separation_best = calculate_separation(weighted_matrix, ideal_best)
separation_worst = calculate_separation(weighted_matrix, ideal_worst)

# Calculate TOPSIS score
topsis_score = separation_worst / (separation_best + separation_worst)

# Add scores to DataFrame and rank models
df["TOPSIS_Score"] = topsis_score
df["Rank"] = df["TOPSIS_Score"].rank(ascending=False)

# Display final ranking
print(df.sort_values(by="TOPSIS_Score", ascending=False))
